In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# табличка с погодой:

Мы взяли данные о погоде в Москве (метеостанция 65518, в районе ВДНХ) в ноябре 2014 - августе 2015 с сайта https://rp5.ru/Архив_погоды_в_По . Из полученного датасета мы используем следующие признаки:

- Дата
- Температура воздуха в градусах Цельсия (T)
- Атмосферное давление в миллиметрах ртутного столба (Po)
- Относительная влажность в % (U)
- Направление ветра - категориальный признак (DD)
- Скорость ветра в метрах в секунду (Ff)
- Общая облачность (N)
- Горизонтальная дальность видимости в км (VV)
- Температура точки росы в градусах Цельсия (Td)

In [2]:
df = pd.read_csv('погода1.csv', sep = ';', header = 1)[['Местное время в По', 'T', 'Po', 'U', 'DD', 'Ff', 'N', 'VV', 'Td']]
df.head(3)

,Местное время в По,T,Po,U,DD,Ff,N,VV,Td
0,31.08.2015 21:00,23.1,731.9,94.0,"Ветер, дующий с юго-юго-запада",2,"90 или более, но не 100%",10,22.1
1,31.08.2015 18:00,28.6,731.0,83.0,"Ветер, дующий с юго-юго-востока",1,"90 или более, но не 100%",10,25.4
2,31.08.2015 15:00,30.0,730.9,72.0,"Ветер, дующий с юго-юго-запада",2,"90 или более, но не 100%",10,24.4


In [3]:
df.dtypes

Местное время в По     object
T                     float64
Po                    float64
U                     float64
DD                     object
Ff                      int64
N                      object
VV                     object
Td                    float64
dtype: object

Посмотрим, какие занчения принимают категориальные признаки

In [4]:
np.unique(df.DD)

array(['Ветер, дующий с востока',
       'Ветер, дующий с востоко-северо-востока',
       'Ветер, дующий с востоко-юго-востока', 'Ветер, дующий с запада',
       'Ветер, дующий с западо-северо-запада',
       'Ветер, дующий с западо-юго-запада', 'Ветер, дующий с севера',
       'Ветер, дующий с северо-востока', 'Ветер, дующий с северо-запада',
       'Ветер, дующий с северо-северо-востока',
       'Ветер, дующий с северо-северо-запада', 'Ветер, дующий с юга',
       'Ветер, дующий с юго-востока', 'Ветер, дующий с юго-запада',
       'Ветер, дующий с юго-юго-востока',
       'Ветер, дующий с юго-юго-запада', 'Штиль, безветрие'], dtype=object)

In [5]:
np.unique(df.N)

array(['100%.', '20–30%.', '40%.', '60%.', '90  или более, но не 100%',
       'Облаков нет.'], dtype=object)

In [6]:
np.unique(df.VV)

array(['0.4', '0.7', '0.8', '0.9', '1', '1.2', '1.4', '1.8', '10', '15',
       '18', '2', '2.5', '2.8', '20', '3.2', '3.5', '4', '5', '6', '8',
       'менее 0.05'], dtype=object)

Признак N (общая облачность) упорядочить:

In [7]:
mapper = {'Облаков нет.': 0 , '20–30%.': 1, '40%.': 2, '60%.': 3,  '90  или более, но не 100%': 4, '100%.': 5}

In [8]:
df.N = df.N.apply(lambda x: mapper[x])

Признак VV (горизонтальная дальность видимости) мы решили привести к вещественному виду. Наблюдениям со значением 'менее 0.05' мы даем значение 0.05.

In [9]:
def realizer(x):
    if x.split()[0].isalpha():
        return float(x.split()[1])
    else:
        return float(x)

In [10]:
df.VV = df.VV.apply(realizer)

In [11]:
df.head()

,Местное время в По,T,Po,U,DD,Ff,N,VV,Td
0,31.08.2015 21:00,23.1,731.9,94.0,"Ветер, дующий с юго-юго-запада",2,4,10.0,22.1
1,31.08.2015 18:00,28.6,731.0,83.0,"Ветер, дующий с юго-юго-востока",1,4,10.0,25.4
2,31.08.2015 15:00,30.0,730.9,72.0,"Ветер, дующий с юго-юго-запада",2,4,10.0,24.4
3,31.08.2015 12:00,29.4,732.4,75.0,"Ветер, дующий с юго-запада",2,4,10.0,24.6
4,31.08.2015 9:00,26.5,732.7,84.0,"Ветер, дующий с юго-запада",2,4,10.0,23.6


В нашем датасате с погодой для каждого дня присутствуют наблюдения для нескольких временных точек. Мы ограничимся использованием четырех: 9:00, 12:00, 15:00, 18:00. Создадим для них отдельные таблицы

#### 9 часов:

In [12]:
df['Местное время в По'] = pd.to_datetime(df['Местное время в По'], format='%d.%m.%Y %H:%M')
df = df.rename(columns={'Местное время в По': 'date'})

# Фильтрация данных, чтобы оставить только записи со временем 09:00
df_9 = df[df['date'].dt.hour == 9]
df_9 = pd.DataFrame(df_9)
df_9['date'] = df_9['date'].dt.date

df_9.sample(3)

,date,T,Po,U,DD,Ff,N,VV,Td
635,2015-06-12,29.0,731.8,75.0,"Ветер, дующий с западо-юго-запада",3,4,10.0,24.1
918,2015-05-07,33.4,730.0,52.0,"Ветер, дующий с запада",4,4,10.0,22.1
2065,2014-12-05,27.8,731.8,28.0,"Ветер, дующий с востока",2,4,10.0,7.6


#### 12 часов:

In [13]:
df_12 = df[df['date'].dt.hour == 12]
df_12 = pd.DataFrame(df_12)
df_12['date'] = df_12['date'].dt.date

#### 15 часов:

In [14]:
df_15 = df[df['date'].dt.hour == 15]
df_15 = pd.DataFrame(df_15)
df_15['date'] = df_15['date'].dt.date

#### 18 часов:

In [15]:
df_18 = df[df['date'].dt.hour == 18]
df_18 = pd.DataFrame(df_18)
df_18['date'] = df_18['date'].dt.date

### Соединяем данные

Мы объединяем нашу табличку с постами с табличкой с погодой

In [16]:
data = pd.read_csv('lemmatized_no_noise.csv').drop('Unnamed: 0', axis = 1)
data.sample(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text
8482,dying,2015-01-08,478,2,False,False,окей гугл находить идеальный парень
1493,dying,2015-07-13,554,10,True,False,рассказывать история показывать
4434,dying,2015-04-27,1656,30,False,False,любовь настя


In [17]:
data.isna().sum()

author         0
date           0
n_likes        0
n_reposts      0
with_photo     0
is_repost?     0
text          24
dtype: int64

In [18]:
data.sample(3)

,author,date,n_likes,n_reposts,with_photo,is_repost?,text
6318,dying,2015-03-06,1124,10,False,False,рядом сразу накрасить высыпаться голова помыть...
9043,dying,2014-12-21,414,1,False,False,главный проблема декабрь 1 сдыхать время сесси...
2404,dying,2015-06-19,999,15,False,False,находить алина че лох алина


In [19]:
data.date = data.date.apply(str) # для простоты мерджа, переведем все даты в формат строки

df_9.date = df_9.date.apply(str)
df_12.date = df_12.date.apply(str)
df_15.date = df_15.date.apply(str)
df_18.date = df_18.date.apply(str)

In [20]:
merged_9 = pd.merge(left = data, right = df_9, on = 'date', how = 'left')
merged_12 = pd.merge(left = data, right = df_12, on = 'date', how = 'left')
merged_15 = pd.merge(left = data, right = df_15, on = 'date', how = 'left')
merged_18 = pd.merge(left = data, right = df_18, on = 'date', how = 'left')

In [21]:
merged_9.to_csv('data_weather_9.csv')
merged_12.to_csv('data_weather_12.csv')
merged_15.to_csv('data_weather_15.csv')
merged_18.to_csv('data_weather_18.csv')

In [22]:
df['T'].value_counts()

26.0    49
27.0    45
28.0    40
30.0    39
29.0    34
        ..
17.8     1
41.8     1
17.5     1
15.9     1
19.3     1
Name: T, Length: 239, dtype: int64

In [23]:
df['T'].max()

41.8

In [24]:
df['T'].min()

15.0

# пытаемся анализировать тональность текста постов:

In [25]:
!pip install textblob

In [26]:
from textblob import TextBlob

# Определение функции для определения тональности текста
def get_sentiment(text):
    if isinstance(text, str):  # Проверка на тип значения
        blob = TextBlob(text)
        sentiment = blob.sentiment.polarity
        if sentiment > 0:
            return 'Положительная'
        elif sentiment < 0:
            return 'Отрицательная'
        else:
            return 'Нейтральная'
    else:
        return 'Некорректный тип'

data['тональность'] = data['text'].apply(get_sentiment)
data

,author,date,n_likes,n_reposts,with_photo,is_repost?,text,тональность
0,dying,2015-08-17,14351,325,False,False,хотеть видеть каждый день,Нейтральная
1,dying,2015-08-17,7098,144,False,False,жить жить твой жизнь появляться полина,Нейтральная
2,dying,2015-08-17,6439,121,False,False,скучать наташа,Нейтральная
3,dying,2015-08-17,8469,221,False,False,давать бог каждый сердце искать,Нейтральная
4,dying,2015-08-17,3704,71,False,False,арина милашка арина бандитка арина псих арина ...,Нейтральная
...,...,...,...,...,...,...,...,...
9804,dying,2014-10-30,263,7,False,False,земля носить гравитация тыры пыр,Нейтральная
9805,dying,2014-10-30,67,0,False,False,сохранять нежность,Нейтральная
9806,dying,2014-10-30,133,4,False,False,лицо прыщик комплексовать подумать вместо сиська,Нейтральная
9807,dying,2014-10-30,636,12,False,False,милый пара видеть держаться рука держать рука ...,Нейтральная


In [27]:
data['тональность'].value_counts()

Нейтральная         9772
Некорректный тип      24
Положительная         10
Отрицательная          3
Name: тональность, dtype: int64

In [28]:
data_0 = pd.read_csv('data.csv')

In [29]:
data_0['тональность'] = data_0['text'].apply(get_sentiment)
data_0

,Unnamed: 0,author,date,text,n_likes,n_reposts,with_photo,is_repost?,тональность
0,0,dying,17 авг 2015,хочу видеть тебя каждый день,14351,325,False,False,Нейтральная
1,1,dying,17 авг 2015,"Живёшь живёшь, и тут в твоей жизни появляется ...",7098,144,False,False,Нейтральная
2,2,dying,17 авг 2015,- По кому скучаешь?\n- По Наташе,6439,121,False,False,Нейтральная
3,3,dying,17 авг 2015,"и дай Бог каждому быть с тем, с кем сердце не ...",8469,221,False,False,Нейтральная
4,4,dying,17 авг 2015,Моя Арина милашка\nМоя Арина бандитка\nМоя Ари...,3704,71,False,False,Нейтральная
...,...,...,...,...,...,...,...,...,...
9984,9987,dying,30 окт 2014,"- Как тебя Земля носит?\n- Ну, гравитация и ты...",263,7,False,False,Нейтральная
9985,9988,dying,30 окт 2014,сохрани нежность,67,0,False,False,Нейтральная
9986,9989,dying,30 окт 2014,"Если у тебя на лице прыщики, не комплексуй. По...",133,4,False,False,Нейтральная
9987,9990,dying,30 окт 2014,"- Вы такая милая пара! Когда я вас вижу, вы вс...",636,12,False,False,Нейтральная


In [30]:
data_0['тональность'].value_counts()

Нейтральная         9692
Некорректный тип     180
Положительная        111
Отрицательная          6
Name: тональность, dtype: int64

In [31]:
translated = pd.read_excel('перевод.xlsx')
translated

,Unnamed: 0,Unnamed: 0.1,author,date,n_likes,n_reposts,with_photo,is_repost?,text,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,0,0,dying,2015-08-17,14351,325,False,False,хотеть видеть каждый день,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,want to see every day
1,1,1,dying,2015-08-17,7098,144,False,False,жить жить твой жизнь появляться полина,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,live live your life appear polina
2,2,2,dying,2015-08-17,6439,121,False,False,скучать наташа,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,miss natasha
3,3,3,dying,2015-08-17,8469,221,False,False,давать бог каждый сердце искать,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,god give every heart to seek
4,4,4,dying,2015-08-17,3704,71,False,False,арина милашка арина бандитка арина псих арина ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,arina cutie arina bandit arina psycho arina good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9804,9804,9804,dying,2014-10-30,263,7,False,False,земля носить гравитация тыры пыр,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,earth wear gravity tyry pyr
9805,9805,9805,dying,2014-10-30,67,0,False,False,сохранять нежность,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,keep tenderness
9806,9806,9806,dying,2014-10-30,133,4,False,False,лицо прыщик комплексовать подумать вместо сиська,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,face pimple complex think instead of boob
9807,9807,9807,dying,2014-10-30,636,12,False,False,милый пара видеть держаться рука держать рука ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cute couple see hold hand hold hand ass pinching


Приводим файл к нормальному виду.

In [32]:
translated = translated.drop(['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [34]:
translated = translated.rename(columns={'Unnamed: 18': 'translation'})
translated

,author,date,n_likes,n_reposts,with_photo,is_repost?,text,translation
0,dying,2015-08-17,14351,325,False,False,хотеть видеть каждый день,want to see every day
1,dying,2015-08-17,7098,144,False,False,жить жить твой жизнь появляться полина,live live your life appear polina
2,dying,2015-08-17,6439,121,False,False,скучать наташа,miss natasha
3,dying,2015-08-17,8469,221,False,False,давать бог каждый сердце искать,god give every heart to seek
4,dying,2015-08-17,3704,71,False,False,арина милашка арина бандитка арина псих арина ...,arina cutie arina bandit arina psycho arina good
...,...,...,...,...,...,...,...,...
9804,dying,2014-10-30,263,7,False,False,земля носить гравитация тыры пыр,earth wear gravity tyry pyr
9805,dying,2014-10-30,67,0,False,False,сохранять нежность,keep tenderness
9806,dying,2014-10-30,133,4,False,False,лицо прыщик комплексовать подумать вместо сиська,face pimple complex think instead of boob
9807,dying,2014-10-30,636,12,False,False,милый пара видеть держаться рука держать рука ...,cute couple see hold hand hold hand ass pinching


In [35]:
translated.to_csv('translated posts.csv')

In [36]:
translated['тональность'] = translated['translation'].apply(get_sentiment)
translated

,author,date,n_likes,n_reposts,with_photo,is_repost?,text,translation,тональность
0,dying,2015-08-17,14351,325,False,False,хотеть видеть каждый день,want to see every day,Нейтральная
1,dying,2015-08-17,7098,144,False,False,жить жить твой жизнь появляться полина,live live your life appear polina,Положительная
2,dying,2015-08-17,6439,121,False,False,скучать наташа,miss natasha,Нейтральная
3,dying,2015-08-17,8469,221,False,False,давать бог каждый сердце искать,god give every heart to seek,Нейтральная
4,dying,2015-08-17,3704,71,False,False,арина милашка арина бандитка арина псих арина ...,arina cutie arina bandit arina psycho arina good,Положительная
...,...,...,...,...,...,...,...,...,...
9804,dying,2014-10-30,263,7,False,False,земля носить гравитация тыры пыр,earth wear gravity tyry pyr,Нейтральная
9805,dying,2014-10-30,67,0,False,False,сохранять нежность,keep tenderness,Нейтральная
9806,dying,2014-10-30,133,4,False,False,лицо прыщик комплексовать подумать вместо сиська,face pimple complex think instead of boob,Отрицательная
9807,dying,2014-10-30,636,12,False,False,милый пара видеть держаться рука держать рука ...,cute couple see hold hand hold hand ass pinching,Положительная


In [37]:
translated['тональность'].value_counts()

Нейтральная         4019
Положительная       3730
Отрицательная       2036
Некорректный тип      24
Name: тональность, dtype: int64

In [45]:
# Присвоение числовых значений типам тональности
mapping = {'Положительная': 1, 'Отрицательная': -1, 'Нейтральная': 0}
translated['tonality_numbers'] = translated['тональность'].map(mapping)

corr = translated['n_likes'].corr(translated['tonality_numbers'])
corr

0.003941953389881087

In [48]:
translated['n_likes'].mean()

1106.163727189316

In [50]:
translated[translated.tonality_numbers == 1]['n_likes'].mean()

1106.3195710455764

In [51]:
translated[translated.tonality_numbers == 0]['n_likes'].mean()

1112.3471012689724

In [52]:
translated[translated.tonality_numbers == -1]['n_likes'].mean()

1095.6335952848724